In [227]:
from sklearn.feature_extraction.text import CountVectorizer

# Import data
train = pd.read_csv("train.csv", delimiter = ",")
test = pd.read_csv("test.csv", delimiter = ",")

In [228]:
# Prepare train data
train_nona = train.dropna()
x_train = train_nona['Reviews']
y_train = train_nona['Rating']

# Prepare test data
x_test = test['Reviews']
x_test = x_test.fillna('.')

# Tokenizing reviews and creating feature dict
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [229]:
# Training the model
rfc = RandomForestClassifier(random_state = 111, n_jobs = -1, verbose = True)
rfc.fit(x_train, y_train)

# Testing the model
predictions = rfc.predict(x_test)

C:\Users\jsolo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   32.2s remaining:   21.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   41.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.4s finished


In [230]:
# Exporting results
test['Rating'] = predictions
results = test[['Id','Rating']]
results.to_csv("results.csv", index = False)